In [340]:
import pandas as pd

In [341]:
desmatamento_pib = pd.read_csv('data/PIB/df_desmatamento_pib.csv', sep=';', encoding='utf-8')

In [342]:
desmatamento_pib.columns

Index(['year', 'areakm', 'municipality', 'geocode_ibge', 'state',
       'FullLocation',
       'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)',
       'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)',
       'Valor adicionado bruto da Agropecuária, \na preços correntes\n(R$ 1.000)',
       'Valor adicionado bruto da Indústria,\na preços correntes\n(R$ 1.000)',
       'Valor adicionado bruto dos Serviços,\na preços correntes \n- exceto Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)'],
      dtype='object')

In [343]:
desmatamento_pib = desmatamento_pib.drop(columns=['state'])

desmatamento_pib = desmatamento_pib.rename(columns={
    'year': 'Ano',
    'areakm': 'Desmatamento (km²)',
    'geocode_ibge': 'Código IBGE',
    'municipality': 'Município',
    'FullLocation': 'Localização',
    'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)': 'PIB per capita (R$)',
    'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)': 'PIB (R$ 1.000)',
    'Valor adicionado bruto da Agropecuária, \na preços correntes\n(R$ 1.000)' : 'VAB Agropecuária (R$ 1.000)',
    'Valor adicionado bruto da Indústria,\na preços correntes\n(R$ 1.000)' : 'VAB Indústria (R$ 1.000)',
    'Valor adicionado bruto dos Serviços,\na preços correntes \n- exceto Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)' : 'VAB Serviços (R$ 1.000)',
})
desmatamento_pib = desmatamento_pib[['Município', 'Ano', 'Código IBGE', 'Localização', 'Desmatamento (km²)', 'PIB per capita (R$)', 'PIB (R$ 1.000)', 'VAB Agropecuária (R$ 1.000)', 'VAB Indústria (R$ 1.000)', 'VAB Serviços (R$ 1.000)']]

In [344]:
from sklearn.linear_model import LinearRegression

# Função para extrapolar valores ausentes de PIB per capita por município e o PIB  bruto
def extrapolar_pib(df, coluna='PIB per capita (R$)'):
    df = df.copy()
    for municipio in df['Município'].unique():
        dados = df[df['Município'] == municipio]
        mask = dados[coluna].notna()
        if mask.sum() >= 2:  # precisa de pelo menos 2 pontos para regressão
            X = dados.loc[mask, 'Ano'].values.reshape(-1, 1)
            y = dados.loc[mask, coluna].values
            model = LinearRegression()
            model.fit(X, y)
            anos_faltantes = dados.loc[~mask, 'Ano']
            if not anos_faltantes.empty:
                pred = model.predict(anos_faltantes.values.reshape(-1, 1))
                df.loc[(df['Município'] == municipio) & (df['Ano'].isin(anos_faltantes)), coluna] = pred
    return df

desmatamento_pib = extrapolar_pib(desmatamento_pib, 'PIB per capita (R$)')
desmatamento_pib['PIB per capita (R$)'] = desmatamento_pib['PIB per capita (R$)'].round(2)

desmatamento_pib = extrapolar_pib(desmatamento_pib, 'PIB (R$ 1.000)')
desmatamento_pib['PIB (R$ 1.000)'] = desmatamento_pib['PIB (R$ 1.000)'].round(2)

for coluna in ['VAB Agropecuária (R$ 1.000)', 'VAB Indústria (R$ 1.000)', 'VAB Serviços (R$ 1.000)']:
    desmatamento_pib = extrapolar_pib(desmatamento_pib, coluna)
    desmatamento_pib[coluna] = desmatamento_pib[coluna].round(2)

In [345]:
desmatamento_pib['População'] = (desmatamento_pib['PIB (R$ 1.000)'] / desmatamento_pib['PIB per capita (R$)']) * 1000
desmatamento_pib['População'] = desmatamento_pib['População'].round(0).astype(int)

In [346]:
desmatamento_pib

,Município,Ano,Código IBGE,Localização,Desmatamento (km²),PIB per capita (R$),PIB (R$ 1.000),VAB Agropecuária (R$ 1.000),VAB Indústria (R$ 1.000),VAB Serviços (R$ 1.000),População
0,Abaetetuba,2008,1500107,"Abaetetuba, Pará, Brazil",1.242461,3325.16,458888.70,52808.71,44176.94,152107.26,138005
1,Abaetetuba,2009,1500107,"Abaetetuba, Pará, Brazil",2.989674,3642.24,509254.35,62031.90,30267.13,174853.63,139819
2,Abaetetuba,2010,1500107,"Abaetetuba, Pará, Brazil",3.828983,4138.79,583793.45,71932.59,31239.98,201654.94,141054
3,Abaetetuba,2011,1500107,"Abaetetuba, Pará, Brazil",0.832087,4971.70,709883.61,88434.83,44167.31,247545.10,142785
4,Abaetetuba,2012,1500107,"Abaetetuba, Pará, Brazil",2.240918,5526.11,798053.28,115890.04,36469.75,276895.20,144415
...,...,...,...,...,...,...,...,...,...,...,...
2443,Óbidos,2020,1505106,"Óbidos, Pará, Brazil",42.423582,17167.44,897960.34,419448.00,56388.87,149339.06,52306
2444,Óbidos,2021,1505106,"Óbidos, Pará, Brazil",29.569607,18408.13,965929.86,426321.31,106455.48,148004.56,52473
2445,Óbidos,2022,1505106,"Óbidos, Pará, Brazil",31.668326,18852.65,984523.73,449684.86,71301.47,162168.50,52222
2446,Óbidos,2023,1505106,"Óbidos, Pará, Brazil",23.694052,19906.75,1041401.35,477191.79,75716.97,170475.72,52314


In [347]:
queimadas = pd.read_csv('data/queimadas/focos_br_pa_ref_2008_2024.csv', encoding='utf-8')

In [348]:
queimadas

,municipio,Ano,focos
0,ABAETETUBA,2008,194
1,ABAETETUBA,2009,190
2,ABAETETUBA,2010,162
3,ABAETETUBA,2011,130
4,ABAETETUBA,2012,153
...,...,...,...
2413,ÓBIDOS,2020,612
2414,ÓBIDOS,2021,377
2415,ÓBIDOS,2022,428
2416,ÓBIDOS,2023,1249


In [349]:
desmatamento_pib['municipioUPPERCASE'] = desmatamento_pib['Município'].str.upper()

In [350]:
desmatamento_pib

,Município,Ano,Código IBGE,Localização,Desmatamento (km²),PIB per capita (R$),PIB (R$ 1.000),VAB Agropecuária (R$ 1.000),VAB Indústria (R$ 1.000),VAB Serviços (R$ 1.000),População,municipioUPPERCASE
0,Abaetetuba,2008,1500107,"Abaetetuba, Pará, Brazil",1.242461,3325.16,458888.70,52808.71,44176.94,152107.26,138005,ABAETETUBA
1,Abaetetuba,2009,1500107,"Abaetetuba, Pará, Brazil",2.989674,3642.24,509254.35,62031.90,30267.13,174853.63,139819,ABAETETUBA
2,Abaetetuba,2010,1500107,"Abaetetuba, Pará, Brazil",3.828983,4138.79,583793.45,71932.59,31239.98,201654.94,141054,ABAETETUBA
3,Abaetetuba,2011,1500107,"Abaetetuba, Pará, Brazil",0.832087,4971.70,709883.61,88434.83,44167.31,247545.10,142785,ABAETETUBA
4,Abaetetuba,2012,1500107,"Abaetetuba, Pará, Brazil",2.240918,5526.11,798053.28,115890.04,36469.75,276895.20,144415,ABAETETUBA
...,...,...,...,...,...,...,...,...,...,...,...,...
2443,Óbidos,2020,1505106,"Óbidos, Pará, Brazil",42.423582,17167.44,897960.34,419448.00,56388.87,149339.06,52306,ÓBIDOS
2444,Óbidos,2021,1505106,"Óbidos, Pará, Brazil",29.569607,18408.13,965929.86,426321.31,106455.48,148004.56,52473,ÓBIDOS
2445,Óbidos,2022,1505106,"Óbidos, Pará, Brazil",31.668326,18852.65,984523.73,449684.86,71301.47,162168.50,52222,ÓBIDOS
2446,Óbidos,2023,1505106,"Óbidos, Pará, Brazil",23.694052,19906.75,1041401.35,477191.79,75716.97,170475.72,52314,ÓBIDOS


In [351]:
desmatamento_pib_queimadas = pd.merge(desmatamento_pib, queimadas, how='left', left_on=['Ano', 'municipioUPPERCASE'], right_on=['Ano', 'municipio'])

In [352]:
desmatamento_pib_queimadas.drop(columns=['municipioUPPERCASE', 'municipio'], inplace=True)
desmatamento_pib_queimadas.rename(columns={'focos': 'Focos de Queimada'}, inplace=True)
desmatamento_pib_queimadas['Focos de Queimada'] = desmatamento_pib_queimadas['Focos de Queimada'].fillna(0)
desmatamento_pib_queimadas['Focos de Queimada'] = desmatamento_pib_queimadas['Focos de Queimada'].astype(int)

In [353]:
desmatamento_pib_queimadas.to_csv('data/RESULTADOS/df_desmatamento_pib_queimadas.csv', sep=',', index=False, encoding='utf-8')

In [354]:
bolsa_familia = pd.read_csv('data/BOLSAFAMILIA/visdata3-download-11-05-2025 133427.csv', encoding='latin1')

In [355]:
bolsa_familia.columns

Index(['Código', 'Unidade Territorial', 'UF', 'Referência',
       'Total de Benefícios Básicos', 'Total de Benefícios Variáveis',
       'Total de Benefícios Variáveis vinculados ao Jovem (BVJ)',
       'Total de Benefícios Variáveis vinculados à Nutriz (BVN)',
       'Total de Benefícios Variáveis vinculados à Gestante (BVG)',
       'Total de Benefícios para Superação da Extrema Pobreza (BSP)'],
      dtype='object')

In [356]:
bolsa_familia = bolsa_familia[['Código', 'Unidade Territorial', 'Referência', 'Total de Benefícios Básicos']]

In [357]:
bolsa_familia = bolsa_familia.rename(columns={
    'Código': 'Código IBGE',
    'Unidade Territorial': 'Município',
    'Referência': 'Ano',
    'Total de Benefícios Básicos': 'Total de Benefícios Básicos (Bolsa Família)',
})

In [358]:
desmatamento_pib_queimadas

,Município,Ano,Código IBGE,Localização,Desmatamento (km²),PIB per capita (R$),PIB (R$ 1.000),VAB Agropecuária (R$ 1.000),VAB Indústria (R$ 1.000),VAB Serviços (R$ 1.000),População,Focos de Queimada
0,Abaetetuba,2008,1500107,"Abaetetuba, Pará, Brazil",1.242461,3325.16,458888.70,52808.71,44176.94,152107.26,138005,194
1,Abaetetuba,2009,1500107,"Abaetetuba, Pará, Brazil",2.989674,3642.24,509254.35,62031.90,30267.13,174853.63,139819,190
2,Abaetetuba,2010,1500107,"Abaetetuba, Pará, Brazil",3.828983,4138.79,583793.45,71932.59,31239.98,201654.94,141054,162
3,Abaetetuba,2011,1500107,"Abaetetuba, Pará, Brazil",0.832087,4971.70,709883.61,88434.83,44167.31,247545.10,142785,130
4,Abaetetuba,2012,1500107,"Abaetetuba, Pará, Brazil",2.240918,5526.11,798053.28,115890.04,36469.75,276895.20,144415,153
...,...,...,...,...,...,...,...,...,...,...,...,...
2443,Óbidos,2020,1505106,"Óbidos, Pará, Brazil",42.423582,17167.44,897960.34,419448.00,56388.87,149339.06,52306,612
2444,Óbidos,2021,1505106,"Óbidos, Pará, Brazil",29.569607,18408.13,965929.86,426321.31,106455.48,148004.56,52473,377
2445,Óbidos,2022,1505106,"Óbidos, Pará, Brazil",31.668326,18852.65,984523.73,449684.86,71301.47,162168.50,52222,428
2446,Óbidos,2023,1505106,"Óbidos, Pará, Brazil",23.694052,19906.75,1041401.35,477191.79,75716.97,170475.72,52314,1249


In [359]:
bolsa_familia

,Código IBGE,Município,Ano,Total de Benefícios Básicos (Bolsa Família)
0,150010,ABAETETUBA,2008,12404
1,150010,ABAETETUBA,2016,21586
2,150010,ABAETETUBA,2015,19325
3,150010,ABAETETUBA,2014,20707
4,150010,ABAETETUBA,2012,20719
...,...,...,...,...
2577,150840,XINGUARA,2016,3166
2578,150840,XINGUARA,2021,3600
2579,150840,XINGUARA,2006,1607
2580,150840,XINGUARA,2004,710


In [360]:
desmatamento_pib_queimadas['MunicípioUPPERCASE'] = desmatamento_pib_queimadas['Município'].str.upper()

In [361]:
desmatamento_pib_queimadas_bolsa = pd.merge(
    desmatamento_pib_queimadas,
    bolsa_familia,
    how='left',
    left_on=['Ano', 'MunicípioUPPERCASE'],
    right_on=['Ano', 'Município'],
    suffixes=('', '_bolsa')
    
)

In [362]:
desmatamento_pib_queimadas_bolsa = desmatamento_pib_queimadas_bolsa.drop(columns=['MunicípioUPPERCASE', 'Código IBGE_bolsa', 'Município_bolsa'])

desmatamento_pib_queimadas_bolsa = extrapolar_pib(desmatamento_pib_queimadas_bolsa, 'Total de Benefícios Básicos (Bolsa Família)')
desmatamento_pib_queimadas_bolsa['Total de Benefícios Básicos (Bolsa Família)'] = desmatamento_pib_queimadas_bolsa['Total de Benefícios Básicos (Bolsa Família)'].fillna(0)
desmatamento_pib_queimadas_bolsa['Total de Benefícios Básicos (Bolsa Família)'] = desmatamento_pib_queimadas_bolsa['Total de Benefícios Básicos (Bolsa Família)'].round(0).astype(int)

In [363]:
desmatamento_pib_queimadas_bolsa

,Município,Ano,Código IBGE,Localização,Desmatamento (km²),PIB per capita (R$),PIB (R$ 1.000),VAB Agropecuária (R$ 1.000),VAB Indústria (R$ 1.000),VAB Serviços (R$ 1.000),População,Focos de Queimada,Total de Benefícios Básicos (Bolsa Família)
0,Abaetetuba,2008,1500107,"Abaetetuba, Pará, Brazil",1.242461,3325.16,458888.70,52808.71,44176.94,152107.26,138005,194,12404
1,Abaetetuba,2009,1500107,"Abaetetuba, Pará, Brazil",2.989674,3642.24,509254.35,62031.90,30267.13,174853.63,139819,190,14743
2,Abaetetuba,2010,1500107,"Abaetetuba, Pará, Brazil",3.828983,4138.79,583793.45,71932.59,31239.98,201654.94,141054,162,16672
3,Abaetetuba,2011,1500107,"Abaetetuba, Pará, Brazil",0.832087,4971.70,709883.61,88434.83,44167.31,247545.10,142785,130,18705
4,Abaetetuba,2012,1500107,"Abaetetuba, Pará, Brazil",2.240918,5526.11,798053.28,115890.04,36469.75,276895.20,144415,153,20719
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,Óbidos,2020,1505106,"Óbidos, Pará, Brazil",42.423582,17167.44,897960.34,419448.00,56388.87,149339.06,52306,612,6374
2444,Óbidos,2021,1505106,"Óbidos, Pará, Brazil",29.569607,18408.13,965929.86,426321.31,106455.48,148004.56,52473,377,6148
2445,Óbidos,2022,1505106,"Óbidos, Pará, Brazil",31.668326,18852.65,984523.73,449684.86,71301.47,162168.50,52222,428,6681
2446,Óbidos,2023,1505106,"Óbidos, Pará, Brazil",23.694052,19906.75,1041401.35,477191.79,75716.97,170475.72,52314,1249,6810


### Plantações

In [364]:
soja_pa = pd.read_csv('data/PLANTACOES/soja_pa.csv', encoding='utf-8')
milho_pa = pd.read_csv('data/PLANTACOES/milho_pa.csv', encoding='utf-8')

In [365]:
soja_pa

,Município,Ano,Área plantada (ha)
0,Abaetetuba,2008,0
1,Abaetetuba,2009,0
2,Abaetetuba,2010,0
3,Abaetetuba,2011,0
4,Abaetetuba,2012,0
...,...,...,...
2283,Óbidos,2019,0
2284,Óbidos,2020,0
2285,Óbidos,2021,0
2286,Óbidos,2022,0


In [366]:
milho_pa

,Município,Ano,Área plantada (ha)
0,Abaetetuba,2008,50
1,Abaetetuba,2009,50
2,Abaetetuba,2010,80
3,Abaetetuba,2011,100
4,Abaetetuba,2012,150
...,...,...,...
2203,Óbidos,2019,700
2204,Óbidos,2020,100
2205,Óbidos,2021,100
2206,Óbidos,2022,100


In [367]:
# Adiciona área plantada de soja
desmatamento_pib_queimadas_bolsa = pd.merge(
    desmatamento_pib_queimadas_bolsa,
    soja_pa.rename(columns={'Área plantada (ha)': 'Área plantada soja (ha)'}),
    how='left',
    on=['Município', 'Ano']
)

# Adiciona área plantada de milho
desmatamento_pib_queimadas_bolsa = pd.merge(
    desmatamento_pib_queimadas_bolsa,
    milho_pa.rename(columns={'Área plantada (ha)': 'Área plantada milho (ha)'}),
    how='left',
    on=['Município', 'Ano']
)

# Preenche valores ausentes com 0
desmatamento_pib_queimadas_bolsa = extrapolar_pib(desmatamento_pib_queimadas_bolsa, 'Área plantada soja (ha)')
desmatamento_pib_queimadas_bolsa['Área plantada soja (ha)'] = desmatamento_pib_queimadas_bolsa['Área plantada soja (ha)'].fillna(0)
desmatamento_pib_queimadas_bolsa['Área plantada soja (ha)'] = desmatamento_pib_queimadas_bolsa['Área plantada soja (ha)'].round(0).astype(int)
# Valores menores que 0 são ajustados para 0
desmatamento_pib_queimadas_bolsa.loc[desmatamento_pib_queimadas_bolsa['Área plantada soja (ha)'] < 0, 'Área plantada soja (ha)'] = 0

desmatamento_pib_queimadas_bolsa = extrapolar_pib(desmatamento_pib_queimadas_bolsa, 'Área plantada milho (ha)')
desmatamento_pib_queimadas_bolsa['Área plantada milho (ha)'] = desmatamento_pib_queimadas_bolsa['Área plantada milho (ha)'].fillna(0)
desmatamento_pib_queimadas_bolsa['Área plantada milho (ha)'] = desmatamento_pib_queimadas_bolsa['Área plantada milho (ha)'].round(0).astype(int)
# Valores menores que 0 são ajustados para 0
desmatamento_pib_queimadas_bolsa.loc[desmatamento_pib_queimadas_bolsa['Área plantada milho (ha)'] < 0, 'Área plantada milho (ha)'] = 0

### Rebanho Bovino

In [369]:
bovino_pa = pd.read_csv('data/REBANHO/bovino_pa.csv', encoding='utf-8')

In [370]:
bovino_pa

,Município,Ano,Total Rebanho (Bovino)
0,Abaetetuba,2008,4194
1,Abaetetuba,2009,3438
2,Abaetetuba,2010,2263
3,Abaetetuba,2011,1527
4,Abaetetuba,2012,1859
...,...,...,...
2299,Óbidos,2019,143068
2300,Óbidos,2020,149900
2301,Óbidos,2021,160052
2302,Óbidos,2022,172430


In [371]:
# Adiciona o total do rebanho bovino
desmatamento_pib_queimadas_bolsa = pd.merge(
    desmatamento_pib_queimadas_bolsa,
    bovino_pa,
    how='left',
    on=['Município', 'Ano']
)

# Preenche valores ausentes com 0 e ajusta tipo
desmatamento_pib_queimadas_bolsa = extrapolar_pib(desmatamento_pib_queimadas_bolsa, 'Total Rebanho (Bovino)')
desmatamento_pib_queimadas_bolsa['Total Rebanho (Bovino)'] = desmatamento_pib_queimadas_bolsa['Total Rebanho (Bovino)'].fillna(0)
desmatamento_pib_queimadas_bolsa['Total Rebanho (Bovino)'] = desmatamento_pib_queimadas_bolsa['Total Rebanho (Bovino)'].round(0).astype(int)
desmatamento_pib_queimadas_bolsa.loc[desmatamento_pib_queimadas_bolsa['Total Rebanho (Bovino)'] < 0, 'Total Rebanho (Bovino)'] = 0

In [372]:
desmatamento_pib_queimadas_bolsa

,Município,Ano,Código IBGE,Localização,Desmatamento (km²),PIB per capita (R$),PIB (R$ 1.000),VAB Agropecuária (R$ 1.000),VAB Indústria (R$ 1.000),VAB Serviços (R$ 1.000),População,Focos de Queimada,Total de Benefícios Básicos (Bolsa Família),Área plantada soja (ha),Área plantada milho (ha),Total Rebanho (Bovino)
0,Abaetetuba,2008,1500107,"Abaetetuba, Pará, Brazil",1.242461,3325.16,458888.70,52808.71,44176.94,152107.26,138005,194,12404,0,50,4194
1,Abaetetuba,2009,1500107,"Abaetetuba, Pará, Brazil",2.989674,3642.24,509254.35,62031.90,30267.13,174853.63,139819,190,14743,0,50,3438
2,Abaetetuba,2010,1500107,"Abaetetuba, Pará, Brazil",3.828983,4138.79,583793.45,71932.59,31239.98,201654.94,141054,162,16672,0,80,2263
3,Abaetetuba,2011,1500107,"Abaetetuba, Pará, Brazil",0.832087,4971.70,709883.61,88434.83,44167.31,247545.10,142785,130,18705,0,100,1527
4,Abaetetuba,2012,1500107,"Abaetetuba, Pará, Brazil",2.240918,5526.11,798053.28,115890.04,36469.75,276895.20,144415,153,20719,0,150,1859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,Óbidos,2020,1505106,"Óbidos, Pará, Brazil",42.423582,17167.44,897960.34,419448.00,56388.87,149339.06,52306,612,6374,0,100,149900
2444,Óbidos,2021,1505106,"Óbidos, Pará, Brazil",29.569607,18408.13,965929.86,426321.31,106455.48,148004.56,52473,377,6148,0,100,160052
2445,Óbidos,2022,1505106,"Óbidos, Pará, Brazil",31.668326,18852.65,984523.73,449684.86,71301.47,162168.50,52222,428,6681,0,100,172430
2446,Óbidos,2023,1505106,"Óbidos, Pará, Brazil",23.694052,19906.75,1041401.35,477191.79,75716.97,170475.72,52314,1249,6810,0,50,180010
